In [1]:
using Graphs
using Printf
using DataFrames
using CSV
using LinearAlgebra
using Random 
include("functions.jl")
include("evolve.jl")

run_ga (generic function with 1 method)

In [2]:
infile = "data/large.csv";

In [3]:
vars, G, D, P = init_data(infile)

(Variable[Variable(:HW, 4), Variable(:CO, 2), Variable(:EF, 2), Variable(:FA, 4), Variable(:QV, 2), Variable(:PZ, 3), Variable(:LD, 3), Variable(:SM, 3), Variable(:MD, 4), Variable(:PQ, 2)  …  Variable(:SA, 2), Variable(:GO, 4), Variable(:BY, 2), Variable(:PT, 2), Variable(:SB, 2), Variable(:EN, 3), Variable(:KO, 4), Variable(:ZY, 4), Variable(:HY, 3), Variable(:PI, 2)], SimpleDiGraph{Int64}(0, [Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]  …  Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]], [Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]  …  Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]]), [4 1 … 4 1; 1 2 … 1 2; … ; 3 2 … 2 2; 1 2 … 1 2], -483188.93309789035)

In [4]:
N = 30 
G_best = SimpleDiGraph(length(vars))
y_best = -Inf 

start_time = time()  # Record the start time in seconds

while true 
   
    pop0 = randperm(length(vars))
    k2_base = K2Search(pop0)
    G,y = fit(k2_base, vars, D)
    
    if y > y_best 
        G_best = G
        y_best = y
    end 
    
    current_time = time()  # Get the current time in seconds
    elapsed_minutes = (current_time - start_time) / 60  # Calculate elapsed minutes
    
    println("elapsed time: $elapsed_minutes mins")
    
    if elapsed_minutes > N
        println("Exiting the loop after $N minutes.")
        break
    end
end


elapsed time: 4.825316667556763 mins
elapsed time: 9.605666665236155 mins
elapsed time: 15.232299999396007 mins
elapsed time: 20.988133335113524 mins
elapsed time: 27.27305000225703 mins
elapsed time: 33.58919999996821 mins
Exiting the loop after 30 minutes.


In [5]:
y_best

-410214.10220696894

In [6]:
df = CSV.File(infile) |> DataFrame
column_names = names(df)
num_columns = length(column_names)
idx2names = Dict(i => column_names[i] for i in 1:num_columns)


Dict{Int64, String} with 50 entries:
  5  => "QV"
  35 => "VX"
  30 => "AG"
  32 => "LO"
  6  => "PZ"
  45 => "SB"
  4  => "FA"
  13 => "JJ"
  12 => "EV"
  28 => "LP"
  23 => "NV"
  41 => "SA"
  43 => "BY"
  11 => "WA"
  36 => "EM"
  39 => "QJ"
  7  => "LD"
  25 => "VJ"
  34 => "FH"
  50 => "PI"
  2  => "CO"
  10 => "PQ"
  18 => "EJ"
  26 => "ST"
  27 => "CH"
  ⋮  => ⋮

In [7]:
function write_gph(dag::DiGraph, idx2names, filename)
    open(filename, "w") do io
        for edge in edges(dag)
            @printf(io, "%s,%s\n", idx2names[src(edge)], idx2names[dst(edge)])
        end
    end
end

write_gph (generic function with 1 method)

In [8]:
write_gph(G_best, idx2names, "opt_graph3.gph")

In [9]:
G_best

{50, 140} directed simple Int64 graph

In [10]:
using Graphs  # for DiGraph and add_edge!
using TikzGraphs   # for TikZ plot output
using TikzPictures # to save TikZ as PDF

In [11]:
p = plot(G_best, [column_names[i] for i in 1:num_columns]) # create TikZ plot with labels
save(PDF("graph3.pdf"), p) # save TikZ as PDF